In [2]:
# Declare whether you are on Colab or local
colab = True

In [3]:
if colab==True:
  
  #Mount drive
  from google.colab import drive
  drive.mount('/content/drive')

  #set path to data in Google Drive
  data_path = "/content/drive/MyDrive/2022_Analytics Lab Student Projects/Data/All Topics"

  #install required packages
  !pip install duckdb
  !pip install -U sentence-transformers

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.9 MB 16.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 7.1 MB/s 
     |████████████████████████████████| 4.2 MB 59.8 MB/s 
     |████████████████████████████████| 1.2 MB 56.1 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
     |████████████████████████████████| 6.6 MB 70.5 MB/s 
     |████████████████████████████████| 596 kB 58.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=eebba90a904db01330f360e5e7b940d178fa7a657e15e9a1059f9e2ec32e4fed
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found exi

In [4]:
# Import sentence_transformers 
from sentence_transformers import SentenceTransformer, util

In [5]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import duckdb
import sqlalchemy
#%load_ext sql

### Load data

In [6]:
# Sentences (text & id)
sentences = pd.read_csv(data_path+"/Fulltext of Corpus/sentences.csv")
sentences_clean = sentences.loc[~sentences.sentence_type.isin(["EMPTY", "TAG"])]
del sentences
sentences_clean = sentences_clean.drop_duplicates(subset="sentence") # Drop duplicate sentences (we only care about different sentences)
#sentences_clean = sentences_clean[["sentence_id", "sentence"]]

In [7]:
sentences_clean = sentences_clean[["article_id","sentence_id", "sentence"]]

In [8]:
sentences_art1 = sentences_clean.loc[sentences_clean["article_id"]==1]

In [9]:
# Entites downsampled ("name/id")
#with open("/content/drive/MyDrive/Colab_Notebooks/02_HWR/00_data/TWSM_project/embeddings_entitis_ds_ids.pkl", "rb") as input_file:
#  entities_ids = pickle.load(input_file)

In [10]:
if colab==True:
  ontology_syn = pd.read_csv(data_path+"/Synonyms in Ontology/isontology.csv", index_col=0)
else:
  ontology_syn = pd.read_csv("../Data/SynonymsinOntology/isontology.csv", index_col=0)
ontology_syn.shape

(397946, 4)

In [11]:
# Check number of synonyms per entity_id
ent_sy_group = ontology_syn.groupby("entity_id")[["synonym"]].nunique().sort_values(by="synonym", ascending=False)

In [12]:
# Downsample entities with more than 100 synonyms to a 100 synoyms
idx = ent_sy_group[ent_sy_group["synonym"]>100].index.tolist()

dfs = []
for entity in idx:
  dfs.append((ontology_syn.loc[ontology_syn["entity_id"]==entity]).sample(100))
dfs.append(ontology_syn.loc[~ontology_syn["entity_id"].isin(idx)]) # Append dataframe for all entities with less then 100 synonyms to list

# Combine all dataframes 
ontology_syn_ds = pd.concat(dfs).reset_index(drop=True)

In [13]:
# Note that DBeaver must be closed, before you can run this command!
if colab==True:
  con = duckdb.connect(database=data_path+'/Extracted Information of Corpus/isrecon.duckdb', read_only=False)
else:
  con = duckdb.connect(database='../Data/ExtractedInformationofCorpus/isrecon.duckdb', read_only=False)

In [14]:
# Load first article as df
entities_sample_article = con.execute("""
                       SELECT *
                       FROM entities e
                       WHERE e.article_id = 1""").fetchdf()

entities_sample_article.shape

(132, 46)

In [15]:
entities_sample_article.head()

,article_id,para_id,sentence_start,sentence_id,section_title,subsection_title,label,ent_id,level_1,level_2,...,attr_double,attr_stage,attr_type,attr_temporal,attr_assumption,attr_dsr,attr_paired,attr_setting,attr_level,attr_tool
0,1,1_3,322,1_322_354,Introduction,NaN,TOPIC,knowledge repository,domain specific entity,IS topic,...,0,0,0,0,0,0,0,0,0,0
1,1,1_3,467,1_467_502,Introduction,NaN,TOPIC,digital platform,domain specific entity,IS topic,...,0,0,0,0,0,0,0,0,0,0
2,1,1_3,502,1_502_527,Introduction,NaN,TECHNOLOGY,Salesforce.com,domain specific entity,IS technology,...,0,0,0,0,0,0,0,0,0,0
3,1,1_4,530,1_530_551,Introduction,NaN,TOPIC,participatory design,domain specific entity,IS topic,...,0,0,0,0,0,0,0,0,0,0
4,1,1_5,861,1_861_886,Introduction,NaN,PARTICIPANTS,group participant,domain specific entity,study object,...,0,0,0,0,0,0,0,0,1,0


In [16]:
# Load a pre-trained model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [17]:
#List of unique entities from ontology with synonyms with downsampling
entities_syn_unique_ds = ontology_syn_ds.synonym.unique().tolist()

In [18]:
all_sentences = sentences_art1.sentence.tolist()

In [19]:
# sentence ids 
sent_ids = sentences_art1.sentence_id.to_list()

In [20]:
# create function based on semantic_search
def find_sim_sent_ent(model, sentences, entities, sent_ids, top_k = 5):
  #Compute embeddings
  embeddings_sentences = model.encode(sentences, convert_to_tensor=True, show_progress_bar=True)
  embeddings_entities = model.encode(entities, convert_to_tensor=True, show_progress_bar=True)

  #Compute cosine-similarities for each sentence with all entities and return top 5 per sentence
  cosine_scores = util.semantic_search(embeddings_sentences, embeddings_entities, top_k=5)

  #loop over results from semantic_search and create dataframe with sentence id and store in list
  dfs = []
  for idx, sim in enumerate(cosine_scores):
    test = pd.DataFrame(sim)
    test["idx_sent"] = sent_ids[idx]
    test["txt_sent"] = sentences[idx]
    test = test.rename(columns={"corpus_id":"idx_ent"})
    dfs.append(test)

  # concat all dfs 
  dfs_df = pd.concat(dfs)
  dfs_df = dfs_df[["score", "idx_sent", "idx_ent", "txt_sent"]] # reorder columns
  dfs_df[["txt_ent"]] = dfs_df[["idx_ent"]].applymap(lambda x:  entities[x])
  dfs_df = dfs_df.sort_values(by="score", ascending=False, ignore_index=True) # sort by score and reset index

  return dfs_df

In [21]:
%%time
test = find_sim_sent_ent(model, all_sentences, entities_syn_unique_ds, sent_ids, top_k=1)

Batches:   0%|          | 0/18 [00:00<?, ?it/s]

Batches:   0%|          | 0/1112 [00:00<?, ?it/s]

CPU times: user 12.8 s, sys: 1.86 s, total: 14.6 s
Wall time: 23.3 s


In [22]:
pd.set_option('display.max_colwidth', None)

In [23]:
entities_sample_article.head()

,article_id,para_id,sentence_start,sentence_id,section_title,subsection_title,label,ent_id,level_1,level_2,...,attr_double,attr_stage,attr_type,attr_temporal,attr_assumption,attr_dsr,attr_paired,attr_setting,attr_level,attr_tool
0,1,1_3,322,1_322_354,Introduction,NaN,TOPIC,knowledge repository,domain specific entity,IS topic,...,0,0,0,0,0,0,0,0,0,0
1,1,1_3,467,1_467_502,Introduction,NaN,TOPIC,digital platform,domain specific entity,IS topic,...,0,0,0,0,0,0,0,0,0,0
2,1,1_3,502,1_502_527,Introduction,NaN,TECHNOLOGY,Salesforce.com,domain specific entity,IS technology,...,0,0,0,0,0,0,0,0,0,0
3,1,1_4,530,1_530_551,Introduction,NaN,TOPIC,participatory design,domain specific entity,IS topic,...,0,0,0,0,0,0,0,0,0,0
4,1,1_5,861,1_861_886,Introduction,NaN,PARTICIPANTS,group participant,domain specific entity,study object,...,0,0,0,0,0,0,0,0,1,0


In [24]:
sim_scores_art1 = test.merge(entities_sample_article[["sentence_id", "ent_id"]], how="left", left_on="idx_sent", right_on="sentence_id").drop(columns="sentence_id")

In [25]:
ent_syn_list = ontology_syn.synonym.unique().tolist()

In [39]:
sim_scores_art1["correct_match"] = ((sim_scores_art1["txt_ent"].isin(ent_syn_list))&(sim_scores_art1.ent_id.notnull()))#.astype(int)

In [40]:
ranges = np.arange(0.,1.1,0.1)

In [41]:
grouped_df = sim_scores_art1.groupby(pd.cut(sim_scores_art1.score, ranges))[["correct_match"]]
grouped_df.sum()/grouped_df.count()

,correct_match
score,
"(0.0, 0.1]",NaN
"(0.1, 0.2]",NaN
"(0.2, 0.3]",NaN
"(0.3, 0.4]",0.114286
"(0.4, 0.5]",0.196558
"(0.5, 0.6]",0.240324
"(0.6, 0.7]",0.232416
"(0.7, 0.8]",0.348315
"(0.8, 0.9]",0.750000


ValueError: ignored

In [47]:
test.loc[(test["score"]>=0.6)&(test["score"]<=0.95)].tail(20)

,score,idx_sent,idx_ent,txt_sent,txt_ent
417,0.603086,1_10068_10075,28891,The online user community of eBay Germany,electronic marketplaces
418,0.603063,1_6285_6302,24439,"Furthermore, our unique context features and data sources help us mitigate some reverse causality bias.",causality analyses
419,0.602653,1_2898_2918,15747,Such norms of reciprocity may facilitate knowledge contribution (as a form of social exchange) between community members.,contribute knowledge
420,0.602647,1_7912_7921,6549,Values less than 0.001 are rounded to 0.001.,p >= 0 . 01S
421,0.602520,1_1433_1455,11881,"We then investigate the hypotheses using data from the OUC of BMC, a global leader in producing innovative software solutions.",global software developments
422,0.602376,1_11276_11280,6519,0.41 \n0 \n,p of 0 . 01S
423,0.602356,1_7848_7857,32167,Correlations less than 0.01 are rounded to 0.01.,correlation coefficient <
424,0.602176,1_1292_1333,29063,Online communities used by the host firms as marketing instruments to know the thoughts of their product users and influence the product users' purchasing behavior Product users are brought together by common interest and inner admiration for a brand.,online communities
425,0.601971,1_7848_7857,32077,Correlations less than 0.01 are rounded to 0.01.,significant positive correlations
426,0.601557,1_10375_10381,29062,The online user community of Propellerhead,online community
